# Invoke Bedrock model for text generation using zero-shot prompt

> *This notebook should work well with the **`Data Science 3.0`** kernel in SageMaker Studio*

## Introduction

In this notebook we show you how to use a LLM to generate an email response to a customer who provided negative feedback on the quality of customer service that they received from the support engineer. 

We will use Bedrock's Amazon Titan Text large model using the Boto3 API. 

The prompt used in this example is called a zero-shot prompt because we are not providing any examples of text alongside their classification other than the prompt.

**Note:** *This notebook can be run within or outside of AWS environment.*

#### Context
To demonstrate the text generation capability of Amazon Bedrock, we will explore the use of Boto3 client to communicate with Amazon Bedrock API. We will demonstrate different configurations available as well as how simple input can lead to desired outputs.

#### Pattern
We will simply provide the Amazon Bedrock API with an input consisting of a task, an instruction and an input for the model under the hood to generate an output without providing any additional example. The purpose here is to demonstrate how the powerful LLMs easily understand the task at hand and generate compelling outputs.

![](./images/bedrock.jpg)

#### Use case
To demonstrate the generation capability of models in Amazon Bedrock, let's take the use case of email generation.

#### Persona
You are Bob a Customer Service Manager at AnyCompany and some of your customers are not happy with the customer service and are providing negative feedbacks on the service provided by customer support engineers. Now, you would like to respond to those customers humbly aplogizing for the poor service and regain trust. You need the help of an LLM to generate a bulk of emails for you which are human friendly and personalized to the customer's sentiment from previous email correspondence.

#### Implementation
To fulfill this use case, in this notebook we will show how to generate an email with a thank you note based on the customer's previous email.We will use the Amazon Titan Text Large model using the Amazon Bedrock API with Boto3 client. 

## Setup

Before running the rest of this notebook, you'll need to run the cells below to (ensure necessary libraries are installed and) connect to Bedrock.

For more details on how the setup works and ⚠️ **whether you might need to make any changes**, refer to the [Bedrock boto3 setup notebook](../00_Intro/bedrock_boto3_setup.ipynb) notebook.

In [1]:
# Make sure you ran `download-dependencies.sh` from the root of the repository first!
%pip install --no-build-isolation --force-reinstall \
    ../dependencies/awscli-*-py3-none-any.whl \
    ../dependencies/boto3-*-py3-none-any.whl \
    ../dependencies/botocore-*-py3-none-any.whl

%pip install --quiet langchain==0.0.249

Processing /Users/marcasbr/Documents/5-aws-training/immersiondays/genai-oslo/amazon-bedrock-workshop/dependencies/awscli-1.29.21-py3-none-any.whl
Processing /Users/marcasbr/Documents/5-aws-training/immersiondays/genai-oslo/amazon-bedrock-workshop/dependencies/boto3-1.28.21-py3-none-any.whl
Processing /Users/marcasbr/Documents/5-aws-training/immersiondays/genai-oslo/amazon-bedrock-workshop/dependencies/botocore-1.31.21-py3-none-any.whl
  Using cached docutils-0.16-py2.py3-none-any.whl (548 kB)
  Using cached s3transfer-0.6.1-py3-none-any.whl (79 kB)
  Obtaining dependency information for PyYAML<6.1,>=3.10 from https://files.pythonhosted.org/packages/5b/07/10033a403b23405a8fc48975444463d3d10a5c2736b7eb2550b07b367429/PyYAML-6.0.1-cp310-cp310-macosx_11_0_arm64.whl.metadata
  Using cached PyYAML-6.0.1-cp310-cp310-macosx_11_0_arm64.whl.metadata (2.1 kB)
  Using cached colorama-0.4.4-py2.py3-none-any.whl (16 kB)
  Using cached rsa-4.7.2-py3-none-any.whl (34 kB)
  Using cached jmespath-1.0.1-p

In [6]:
import json
import os
import sys

import boto3

module_path = ".."
sys.path.append(os.path.abspath(module_path))
from utils import bedrock, print_ww


# ---- ⚠️ Un-comment and edit the below lines as needed for your AWS setup ⚠️ ----

# os.environ["AWS_DEFAULT_REGION"] = "<REGION_NAME>"  # E.g. "us-east-1"
# os.environ["AWS_PROFILE"] = "<YOUR_PROFILE>"
# os.environ["BEDROCK_ASSUME_ROLE"] = "<YOUR_ROLE_ARN>"  # E.g. "arn:aws:..."
# os.environ["BEDROCK_ENDPOINT_URL"] = "<YOUR_ENDPOINT_URL>"  # E.g. "https://..."
os.environ['AWS_PROFILE'] = 'sengledo+bedrock-benelux-Admin'


boto3_bedrock = bedrock.get_bedrock_client(
    assumed_role=os.environ.get("BEDROCK_ASSUME_ROLE", None),
    endpoint_url=os.environ.get("BEDROCK_ENDPOINT_URL", None),
    region=os.environ.get("AWS_DEFAULT_REGION", None),
)

Create new client
  Using region: us-east-1
  Using profile: sengledo+bedrock-benelux-Admin
boto3 Bedrock client successfully created!
bedrock(https://bedrock.us-east-1.amazonaws.com)


## Generate text

Following on the use case explained above, let's prepare an input for  the Amazon Bedrock service to generate an email

In [7]:
# create the prompt
prompt_data = """Command: Write an email from Bob, Customer Service Manager, to the customer "John Doe" 
who provided negative feedback on the service provided by our customer support 
engineer"""


Let's start by using the Amazon Titan Large model. Amazon Titan Large supports a context window of ~4k tokens and accepts the following parameters:
- `inputText`: Prompt to the LLM
- `textGenerationConfig`: These are the parameters that model will take into account while generating the output.

In [8]:
body = json.dumps({
    "inputText": prompt_data, 
    "textGenerationConfig":{
        "maxTokenCount":4096,
        "stopSequences":[],
        "temperature":0,
        "topP":0.9
        }
    }) 

The Amazon Bedrock API provides you with an API `invoke_model` which accepts the following:
- `modelId`: This is the model ARN for the various foundation models available under Amazon Bedrock
- `accept`: The type of input request
- `contentType`: The content type of the output
- `body`: A json string consisting of the prompt and the configurations

Available text generation models under Amazon Bedrock have the following IDs:
- `amazon.titan-tg1-large`
- `ai21.j2-grande-instruct`
- `ai21.j2-jumbo-instruct`
- `anthropic.claude-instant-v1`
- `anthropic.claude-v1`

#### Invoke the Amazon Titan Large language model

First, we explore how the model generates an output based on the prompt created earlier.

##### Complete Output Generation

In [9]:
modelId = 'amazon.titan-tg1-large' # change this to use a different version from the model provider
accept = 'application/json'
contentType = 'application/json'

response = boto3_bedrock.invoke_model(body=body, modelId=modelId, accept=accept, contentType=contentType)
response_body = json.loads(response.get('body').read())

outputText = response_body.get('results')[0].get('outputText')


In [10]:
# The relevant portion of the response begins after the first newline character
# Below we print the response beginning after the first occurence of '\n'.

email = outputText[outputText.index('\n')+1:]
print_ww(email)


Subject: Response to your feedback

Dear John,

I'm sorry to hear that you had a negative experience with one of our customer support engineers,
Liam. We take customer feedback very seriously, and I would like to apologize for any inconvenience
or frustration this may have caused you.

I want to assure you that we are taking steps to address the issue you raised. We will be conducting
additional training with our team to ensure that we are providing the highest level of service to
our customers.

In addition, we would like to offer you a refund or credit for the service that you paid for. Please
let us know if you would like us to take further action to resolve this matter.

Thank you for bringing this to our attention. We appreciate your feedback and hope to have the
opportunity to serve you better in the future.

Sincerely,

Bob
Customer Service Manager



##### Streaming Output Generation
Above is an example email generated by the Amazon Titan Large model by understanding the input request and using its inherent understanding of the different modalities. This request to the API is synchronous and waits for the entire output to be generated by the model.

Bedrock also supports that the output can be streamed as it is generated by the model in form of chunks. Below is an example of invoking the model with streaming option. `invoke_model_with_response_stream` returns a `ResponseStream` which you can read from.

In [11]:
response = boto3_bedrock.invoke_model_with_response_stream(body=body, modelId=modelId, accept=accept, contentType=contentType)
stream = response.get('body')
output = []
i = 1
if stream:
    for event in stream:
        chunk = event.get('chunk')
        if chunk:
            chunk_obj = json.loads(chunk.get('bytes').decode())
            text = chunk_obj['outputText']
            output.append(text)
            print(f'\t\t\x1b[31m**Chunk {i}**\x1b[0m\n{text}\n')
            i+=1

		**Chunk 1**
 "Liam"
Subject: Response to your feedback

Dear John,

I'm sorry to hear that you had a negative experience with one of our customer support engineers, Liam. We take customer feedback

		**Chunk 2**
 very seriously, and I would like to apologize for any inconvenience or frustration this may have caused you.

I want to assure you that we are taking steps to address the issue you raised. We will be conducting additional training with our team to ensure that we are providing the highest level of service to our customers.

In ad

		**Chunk 3**
dition, we would like to offer you a refund or credit for the service that you paid for. Please let us know if you would like us to take further action to resolve this matter.

Thank you for bringing this to our attention. We appreciate your feedback and hope to have the opportunity to serve you better in the future.

Sincerely,

Bob
Customer Service Manager




The above helps to quickly get output of the model and let the service complete it as you read. This assists in use-cases where there are longer pieces of text that you request the model to generate. You can later combine all the chunks generated to form the complete output and use it for your use-case

In [12]:
print('\t\t\x1b[31m**COMPLETE OUTPUT**\x1b[0m\n')
complete_output = ''.join(output)
print(complete_output)

		**COMPLETE OUTPUT**

 "Liam"
Subject: Response to your feedback

Dear John,

I'm sorry to hear that you had a negative experience with one of our customer support engineers, Liam. We take customer feedback very seriously, and I would like to apologize for any inconvenience or frustration this may have caused you.

I want to assure you that we are taking steps to address the issue you raised. We will be conducting additional training with our team to ensure that we are providing the highest level of service to our customers.

In addition, we would like to offer you a refund or credit for the service that you paid for. Please let us know if you would like us to take further action to resolve this matter.

Thank you for bringing this to our attention. We appreciate your feedback and hope to have the opportunity to serve you better in the future.

Sincerely,

Bob
Customer Service Manager



## Conclusion
You have now experimented with using `boto3` SDK which provides a vanilla exposure to Amazon Bedrock API. Using this API you have seen the use case of generating an email responding to a customer due to their negative feedback.

### Take aways
- Adapt this notebook to experiment with different models available through Amazon Bedrock such as Anthropic Claude and AI21 Labs Jurassic models.
- Change the prompts to your specific usecase and evaluate the output of different models.
- Play with the token length to understand the latency and responsiveness of the service.
- Apply different prompt engineering principles to get better outputs.

## Thank You